In [67]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

In [68]:
test = pd.read_csv('D:\\Kaggle\\HeartDisease_dataset\\test.csv', encoding = 'euc-kr')
train = pd.read_csv('D:\\Kaggle\\HeartDisease_dataset\\train.csv', encoding = 'euc-kr')

In [69]:
sample_sub = pd.read_csv('D:\\Kaggle\\HeartDisease_dataset\\sample_submission.csv', encoding = 'utf-8')

In [70]:
train

,id,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,1,53,1,2,130,197,1,0,152,0,1.2,0,0,2,1
1,2,52,1,3,152,298,1,1,178,0,1.2,1,0,3,1
2,3,54,1,1,192,283,0,0,195,0,0.0,2,1,3,0
3,4,45,0,0,138,236,0,0,152,1,0.2,1,0,2,1
4,5,35,1,1,122,192,0,1,174,0,0.0,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,147,50,1,2,140,233,0,1,163,0,0.6,1,1,3,0
147,148,51,1,2,94,227,0,1,154,1,0.0,2,1,3,1
148,149,69,1,3,160,234,1,0,131,0,0.1,1,1,2,1
149,150,46,1,0,120,249,0,0,144,0,0.8,2,0,3,0


In [71]:
df = train.copy()

In [72]:
# onehot encoding - train

dummy_cols = ['cp', 'sex', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']
df = pd.get_dummies(df, columns = dummy_cols)
df

,id,age,trestbps,chol,thalach,oldpeak,target,cp_0,cp_1,cp_2,...,slope_1,slope_2,ca_0,ca_1,ca_2,ca_3,thal_0,thal_1,thal_2,thal_3
0,1,53,130,197,152,1.2,1,0,0,1,...,0,0,1,0,0,0,0,0,1,0
1,2,52,152,298,178,1.2,1,0,0,0,...,1,0,1,0,0,0,0,0,0,1
2,3,54,192,283,195,0.0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,1
3,4,45,138,236,152,0.2,1,1,0,0,...,1,0,1,0,0,0,0,0,1,0
4,5,35,122,192,174,0.0,1,0,1,0,...,0,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,147,50,140,233,163,0.6,0,0,0,1,...,1,0,0,1,0,0,0,0,0,1
147,148,51,94,227,154,0.0,1,0,0,1,...,0,1,0,1,0,0,0,0,0,1
148,149,69,160,234,131,0.1,1,0,0,0,...,1,0,0,1,0,0,0,0,1,0
149,150,46,120,249,144,0.8,0,1,0,0,...,0,1,1,0,0,0,0,0,0,1


In [73]:
# onehot encoding - test

dummy_cols = ['cp', 'sex', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']
test = pd.get_dummies(test, columns = dummy_cols)
test

,id,age,trestbps,chol,thalach,oldpeak,cp_0,cp_1,cp_2,cp_3,...,slope_2,ca_0,ca_1,ca_2,ca_3,ca_4,thal_0,thal_1,thal_2,thal_3
0,1,57,150,276,112,0.6,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0
1,2,59,170,288,159,0.2,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
2,3,57,150,126,173,0.2,0,0,1,0,...,1,0,1,0,0,0,0,0,0,1
3,4,56,134,409,150,1.9,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,5,71,110,265,130,0.0,0,0,1,0,...,1,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,148,64,130,303,122,2.0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
148,149,43,132,341,136,3.0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
149,150,53,123,282,95,2.0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
150,151,67,152,277,172,0.0,0,0,1,0,...,1,0,1,0,0,0,0,0,1,0


In [74]:
# Scaling - train

num_col = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

scaler = StandardScaler()
df[num_col] = scaler.fit_transform(df[num_col])

In [75]:
# Scaling - test

num_col = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

scaler = StandardScaler()
test[num_col] = scaler.fit_transform(test[num_col])

In [76]:
df

,id,age,trestbps,chol,thalach,oldpeak,target,cp_0,cp_1,cp_2,...,slope_1,slope_2,ca_0,ca_1,ca_2,ca_3,thal_0,thal_1,thal_2,thal_3
0,1,-0.168640,-0.113897,-0.846549,0.058612,0.206190,1,0,0,1,...,0,0,1,0,0,0,0,0,1,0
1,2,-0.281315,1.118560,0.952353,1.170264,0.206190,1,0,0,0,...,1,0,1,0,0,0,0,0,0,1
2,3,-0.055965,3.359390,0.685189,1.897112,-0.902462,0,0,1,0,...,0,1,0,1,0,0,0,0,0,1
3,4,-1.070043,0.334270,-0.151924,0.058612,-0.717686,1,1,0,0,...,1,0,1,0,0,0,0,0,1,0
4,5,-2.196797,-0.562063,-0.935604,0.999240,-0.902462,1,0,1,0,...,0,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,147,-0.506666,0.446311,-0.205356,0.528926,-0.348136,0,0,0,1,...,1,0,0,1,0,0,0,0,0,1
147,148,-0.393991,-2.130644,-0.312222,0.144124,-0.902462,1,0,0,1,...,0,1,0,1,0,0,0,0,0,1
148,149,1.634166,1.566726,-0.187545,-0.839260,-0.810074,1,0,0,0,...,1,0,0,1,0,0,0,0,1,0
149,150,-0.957368,-0.674104,0.079618,-0.283434,-0.163361,0,1,0,0,...,0,1,1,0,0,0,0,0,0,1


In [77]:
df.isna().sum()

id           0
age          0
trestbps     0
chol         0
thalach      0
oldpeak      0
target       0
cp_0         0
cp_1         0
cp_2         0
cp_3         0
sex_0        0
sex_1        0
fbs_0        0
fbs_1        0
restecg_0    0
restecg_1    0
restecg_2    0
exang_0      0
exang_1      0
slope_0      0
slope_1      0
slope_2      0
ca_0         0
ca_1         0
ca_2         0
ca_3         0
thal_0       0
thal_1       0
thal_2       0
thal_3       0
dtype: int64

In [60]:
# # bias를 위한 상수항 추가

# df = sm.add_constant(df, has_constant = "add")
# df

,const,id,age,trestbps,chol,thalach,oldpeak,target,cp_0,cp_1,...,slope_1,slope_2,ca_0,ca_1,ca_2,ca_3,thal_0,thal_1,thal_2,thal_3
0,1.0,1,-0.168640,-0.113897,-0.846549,0.058612,0.206190,1,0,0,...,0,0,1,0,0,0,0,0,1,0
1,1.0,2,-0.281315,1.118560,0.952353,1.170264,0.206190,1,0,0,...,1,0,1,0,0,0,0,0,0,1
2,1.0,3,-0.055965,3.359390,0.685189,1.897112,-0.902462,0,0,1,...,0,1,0,1,0,0,0,0,0,1
3,1.0,4,-1.070043,0.334270,-0.151924,0.058612,-0.717686,1,1,0,...,1,0,1,0,0,0,0,0,1,0
4,1.0,5,-2.196797,-0.562063,-0.935604,0.999240,-0.902462,1,0,1,...,0,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,1.0,147,-0.506666,0.446311,-0.205356,0.528926,-0.348136,0,0,0,...,1,0,0,1,0,0,0,0,0,1
147,1.0,148,-0.393991,-2.130644,-0.312222,0.144124,-0.902462,1,0,0,...,0,1,0,1,0,0,0,0,0,1
148,1.0,149,1.634166,1.566726,-0.187545,-0.839260,-0.810074,1,0,0,...,1,0,0,1,0,0,0,0,1,0
149,1.0,150,-0.957368,-0.674104,0.079618,-0.283434,-0.163361,0,1,0,...,0,1,1,0,0,0,0,0,0,1


In [61]:
# test = sm.add_constant(test, has_constant = "add")
# test

,const,id,age,trestbps,chol,thalach,oldpeak,cp_0,cp_1,cp_2,...,slope_2,ca_0,ca_1,ca_2,ca_3,ca_4,thal_0,thal_1,thal_2,thal_3
0,1.0,1,0.298651,1.094977,0.597244,-1.644839,-0.408938,1,0,0,...,0,0,1,0,0,0,0,1,0,0
1,1.0,2,0.514818,2.260907,0.853086,0.463293,-0.734802,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2,1.0,3,0.298651,1.094977,-2.600776,1.091248,-0.734802,0,0,1,...,1,0,1,0,0,0,0,0,0,1
3,1.0,4,0.190568,0.162233,3.432822,0.059608,0.650120,1,0,0,...,0,0,0,1,0,0,0,0,0,1
4,1.0,5,1.811817,-1.236883,0.362723,-0.837469,-0.897733,0,0,1,...,1,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,1.0,148,1.055234,-0.070953,1.172888,-1.196300,0.731586,1,0,0,...,0,0,0,1,0,0,0,0,1,0
148,1.0,149,-1.214515,0.045640,1.983053,-0.568346,1.546245,1,0,0,...,0,1,0,0,0,0,0,0,0,1
149,1.0,150,-0.133682,-0.479029,0.725165,-2.407355,0.731586,1,0,0,...,0,0,0,1,0,0,0,0,0,1
150,1.0,151,1.379484,1.211570,0.618564,1.046394,-0.897733,0,0,1,...,1,0,1,0,0,0,0,0,1,0


In [108]:
# ## Stepwise selection

# variables = df.columns[:-1].tolist() ## 설명 변수 리스트
# variables.remove('target') # BMI 제거하고
# variables.remove('id') # BMI 제거하고
# print(variables)
 
# y = df['target'] ## 반응 변수
# selected_variables = [] ## 선택된 변수들
# sl_enter = 0.2
# sl_remove = 0.2
 
# sv_per_step = [] ## 각 스텝별로 선택된 변수들
# adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
# steps = [] ## 스텝
# step = 0
# while len(variables) > 0:
#     remainder = list(set(variables) - set(selected_variables))
#     pval = pd.Series(index=remainder) ## 변수의 p-value
#     ## 기존에 포함된 변수와 새로운 변수 하나씩 돌아가면서 
#     ## 선형 모형을 적합한다.
#     for col in remainder: 
#         X = df[selected_variables+[col]]
#         X = sm.add_constant(X)
#         model = sm.OLS(y,X).fit()
#         pval[col] = model.pvalues[col]
 
#     min_pval = pval.min()
#     if min_pval < sl_enter: ## 최소 p-value 값이 기준 값보다 작으면 포함
#         selected_variables.append(pval.idxmin())
#         ## 선택된 변수들에대해서
#         ## 어떤 변수를 제거할지 고른다.
#         while len(selected_variables) > 0:
#             selected_X = df[selected_variables]
#             selected_X = sm.add_constant(selected_X)
#             selected_pval = sm.OLS(y,selected_X).fit().pvalues[1:] ## 절편항의 p-value는 뺀다
#             max_pval = selected_pval.max()
#             if max_pval >= sl_remove: ## 최대 p-value값이 기준값보다 크거나 같으면 제외
#                 remove_variable = selected_pval.idxmax()
#                 selected_variables.remove(remove_variable)
#             else:
#                 break
        
#         step += 1
#         steps.append(step)
#         adj_r_squared = sm.OLS(y,sm.add_constant(df[selected_variables])).fit().rsquared_adj
#         adjusted_r_squared.append(adj_r_squared)
#         sv_per_step.append(selected_variables.copy())
#     else:
#         break

In [109]:
# fig = plt.figure(figsize=(10,10))
# fig.set_facecolor('white')
 
# font_size = 15
# plt.xticks(steps,[f'step {s}\n'+'\n'.join(sv_per_step[i]) for i,s in enumerate(steps)], fontsize=12)
# plt.plot(steps,adjusted_r_squared, marker='o')
    
# plt.ylabel('Adjusted R Squared',fontsize=font_size)
# plt.grid(True)
# plt.show()

In [62]:
# # r에서 logistic으로 선택해서 가져옴 

# selected_variables = ["chol", "oldpeak", "cp_0", "cp_1", "slope_1", "ca_0", "thal_1", "thal_2"]

In [63]:
# feature_columns = df.columns.difference(['target']) # Target column 빼기
# feature_columns = selected_variables

# X = df[feature_columns]
# y = df.target

# print(X.shape, y.shape)

(151, 8) (151,)


In [64]:
# from sklearn.linear_model import LogisticRegression
# clf = LogisticRegression(random_state=0).fit(X, y)
# clf.score(X, y)

0.8874172185430463

In [93]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf =  RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
# clf =  AdaBoostClassifier()

In [94]:
df

,id,age,trestbps,chol,thalach,oldpeak,target,cp_0,cp_1,cp_2,...,slope_1,slope_2,ca_0,ca_1,ca_2,ca_3,thal_0,thal_1,thal_2,thal_3
0,1,-0.168640,-0.113897,-0.846549,0.058612,0.206190,1,0,0,1,...,0,0,1,0,0,0,0,0,1,0
1,2,-0.281315,1.118560,0.952353,1.170264,0.206190,1,0,0,0,...,1,0,1,0,0,0,0,0,0,1
2,3,-0.055965,3.359390,0.685189,1.897112,-0.902462,0,0,1,0,...,0,1,0,1,0,0,0,0,0,1
3,4,-1.070043,0.334270,-0.151924,0.058612,-0.717686,1,1,0,0,...,1,0,1,0,0,0,0,0,1,0
4,5,-2.196797,-0.562063,-0.935604,0.999240,-0.902462,1,0,1,0,...,0,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,147,-0.506666,0.446311,-0.205356,0.528926,-0.348136,0,0,0,1,...,1,0,0,1,0,0,0,0,0,1
147,148,-0.393991,-2.130644,-0.312222,0.144124,-0.902462,1,0,0,1,...,0,1,0,1,0,0,0,0,0,1
148,149,1.634166,1.566726,-0.187545,-0.839260,-0.810074,1,0,0,0,...,1,0,0,1,0,0,0,0,1,0
149,150,-0.957368,-0.674104,0.079618,-0.283434,-0.163361,0,1,0,0,...,0,1,1,0,0,0,0,0,0,1


In [95]:
col = df.columns.tolist()
col.remove('id')
col.remove('target')

In [96]:
X = df[col]
y = df.target

In [97]:
X

,age,trestbps,chol,thalach,oldpeak,cp_0,cp_1,cp_2,cp_3,sex_0,...,slope_1,slope_2,ca_0,ca_1,ca_2,ca_3,thal_0,thal_1,thal_2,thal_3
0,-0.168640,-0.113897,-0.846549,0.058612,0.206190,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0
1,-0.281315,1.118560,0.952353,1.170264,0.206190,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,1
2,-0.055965,3.359390,0.685189,1.897112,-0.902462,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,1
3,-1.070043,0.334270,-0.151924,0.058612,-0.717686,1,0,0,0,1,...,1,0,1,0,0,0,0,0,1,0
4,-2.196797,-0.562063,-0.935604,0.999240,-0.902462,0,1,0,0,0,...,0,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,-0.506666,0.446311,-0.205356,0.528926,-0.348136,0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,1
147,-0.393991,-2.130644,-0.312222,0.144124,-0.902462,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,1
148,1.634166,1.566726,-0.187545,-0.839260,-0.810074,0,0,0,1,0,...,1,0,0,1,0,0,0,0,1,0
149,-0.957368,-0.674104,0.079618,-0.283434,-0.163361,1,0,0,0,0,...,0,1,1,0,0,0,0,0,0,1


In [98]:
clf.fit(X, y) 

RandomForestClassifier(max_depth=5, max_features=1, n_estimators=10)

In [99]:
testcol = test.columns.tolist()
testcol.remove('id')

test2 = test[testcol]
test2

,age,trestbps,chol,thalach,oldpeak,cp_0,cp_1,cp_2,cp_3,sex_0,...,slope_2,ca_0,ca_1,ca_2,ca_3,ca_4,thal_0,thal_1,thal_2,thal_3
0,0.298651,1.094977,0.597244,-1.644839,-0.408938,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
1,0.514818,2.260907,0.853086,0.463293,-0.734802,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
2,0.298651,1.094977,-2.600776,1.091248,-0.734802,0,0,1,0,0,...,1,0,1,0,0,0,0,0,0,1
3,0.190568,0.162233,3.432822,0.059608,0.650120,1,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
4,1.811817,-1.236883,0.362723,-0.837469,-0.897733,0,0,1,0,1,...,1,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,1.055234,-0.070953,1.172888,-1.196300,0.731586,1,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
148,-1.214515,0.045640,1.983053,-0.568346,1.546245,1,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
149,-0.133682,-0.479029,0.725165,-2.407355,0.731586,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
150,1.379484,1.211570,0.618564,1.046394,-0.897733,0,0,1,0,1,...,1,0,1,0,0,0,0,0,1,0


In [100]:
set(testcol) - set(col)

{'ca_4'}

In [101]:
test2.drop(['ca_4'], axis = 1, inplace = True)

C:\Users\user\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [102]:
pred = clf.predict(test2) 

In [103]:
pred

array([0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1],
      dtype=int64)

In [104]:
sample_sub['target'] = pred

In [105]:
sample_sub

,id,target
0,1,0
1,2,1
2,3,1
3,4,0
4,5,1
...,...,...
147,148,0
148,149,0
149,150,0
150,151,1


In [28]:
sample_sub.to_csv('D:\\Kaggle\\HeartDisease_dataset\\submission_1213_randomforest_ALL.csv', encoding = 'euc-kr', index = False)